importeren 

In [2]:
import pandas as pd
import sqlite3 as sql
import warnings
import pyodbc
warnings.simplefilter('ignore')

In [1]:
import os
import pyodbc

# Define the path to your Access database using absolute path
# This ensures there's no ambiguity in the path resolution
access_db_path = os.path.abspath('../../data/raw/AdventureWorks.mdb')
print(f"Attempting to connect to: {access_db_path}")

# Verify the file exists
if not os.path.exists(access_db_path):
    print(f"Error: Database file not found at {access_db_path}")
    # Try to list files in the directory to help troubleshoot
    dir_path = os.path.dirname(access_db_path)
    if os.path.exists(dir_path):
        print(f"Files in {dir_path}:")
        for file in os.listdir(dir_path):
            print(f"  - {file}")
    else:
        print(f"Directory {dir_path} does not exist")
else:
    print(f"Database file found at {access_db_path}")
    
    try:
        # Build the connection string for Access
        access_conn_str = (
            r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
            r"DBQ=" + access_db_path + ";"
        )
        
        # Attempt to establish the connection
        access_conn = pyodbc.connect(access_conn_str)
        print("Connected to Access database successfully!")
        print(access_conn)
        
        # Don't forget to close the connection when done
        access_conn.close()
        
    except pyodbc.Error as e:
        print(f"Error connecting to database: {e}")
        
        # Check if driver is available
        print("\nChecking available ODBC drivers:")
        for driver in pyodbc.drivers():
            print(f"  - {driver}")

Attempting to connect to: c:\Users\veyse\source\DEDS_portfolio\data\raw\AdventureWorks.mdb
Database file found at c:\Users\veyse\source\DEDS_portfolio\data\raw\AdventureWorks.mdb
Connected to Access database successfully!


Connections


In [3]:
connection_aenc = sql.connect('../../data/raw/aenc.sqlite')
print(connection_aenc)

# Define the path to your Access database
access_db_path = '../../data/raw/AdventureWorks.mdb'

# Build the connection string for Access
access_conn_str = (
    r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"Dbq=" + access_db_path + ";"
)

# Establish the connection
access_conn = pyodbc.connect(access_conn_str)
print("Connected to Access database!")
print(access_conn)


# Build the connection string for SQL Server
sql_server_conn_str = (
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=LAPTOP-HLMO0COA\SQLEXPRESS;"
    r"DATABASE=northwind;"
    r"Trusted_Connection=yes;"
)

# Establish the connection
sql_server_conn = pyodbc.connect(sql_server_conn_str)
print("Connected to SQL Server database!")
print(sql_server_conn)



Connected to Access database!
Connected to SQL Server database!


Legen van Tabbelen

In [3]:


# Connect to the database
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-HLMO0COA\\SQLEXPRESS;' 
    'DATABASE=DWH_project;' 
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

# Get all tables with constraints
tables_query = """
SELECT DISTINCT OBJECT_NAME(parent_object_id) AS TableName
FROM sys.foreign_keys
"""

try:
    cursor.execute(tables_query)
    tables_with_constraints = [row[0] for row in cursor.fetchall()]
    
    # Disable constraints for each table individually
    for table in tables_with_constraints:
        print(f"  Disabling constraints for {table}...")
        cursor.execute(f"ALTER TABLE [{table}] NOCHECK CONSTRAINT ALL")
        conn.commit()
    
    print("All constraints disabled")
except Exception as e:
    print(f"Error disabling constraints: {str(e)}")
    # Continue anyway - we'll still try to import data

# Get all tables
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
all_tables = [row[0] for row in cursor.fetchall()]

# Empty each table
for table_name in all_tables:
    print(f" Legen van tabel: {table_name}")
    try:
        cursor.execute(f"DELETE FROM [{table_name}];")
        conn.commit()
    except Exception as e:
        print(f"Error deleting from {table_name}: {str(e)}")

# Re-enable constraints
try:
    # Re-enable constraints for each table individually
    for table in tables_with_constraints:
        print(f"  Re-enabling constraints for {table}...")
        cursor.execute(f"ALTER TABLE [{table}] WITH CHECK CHECK CONSTRAINT ALL")
        conn.commit()
    
    print("All constraints re-enabled successfully")
except Exception as e:
    print(f"Error re-enabling constraints: {str(e)}")

print(" Alle tabellen in de database 'sdm' zijn geleegd.")

# Close connections at the very end
cursor.close()
conn.close()

  Disabling constraints for Bonus...
  Disabling constraints for Customer...
  Disabling constraints for EmployeeTerritories...
  Disabling constraints for OrderDetails...
  Disabling constraints for Orders...
  Disabling constraints for purchaseOrderDetail...
  Disabling constraints for purchaseOrderHeader...
  Disabling constraints for Sales_Order...
  Disabling constraints for sales_order_item...
  Disabling constraints for Sales_OrderDetail...
  Disabling constraints for Sales_OrderHeader...
  Disabling constraints for Sales_Store...
  Disabling constraints for Territories...
All constraints disabled
 Legen van tabel: Address
 Legen van tabel: billOfMaterials
 Legen van tabel: Bonus
 Legen van tabel: Categories
 Legen van tabel: Customer
 Legen van tabel: Department
 Legen van tabel: Employee
 Legen van tabel: EmployeeTerritories
 Legen van tabel: OrderDetails
 Legen van tabel: Orders
 Legen van tabel: person
 Legen van tabel: Product
 Legen van tabel: productCategory
 Legen van ta

Sqlite importeren


In [4]:
def convert_numpy_type(val):
    if pd.isnull(val):
        return None
    elif hasattr(val, 'item'):  # Handles numpy types (int64, float64, etc.)
        return val.item()
    else:
        return val

# Helper function to handle type conversions
def convert_value_for_column(value, column_name, column_type, table_name):
    """Converts a value to the appropriate type for the target column."""
    if value is None:
        return None
    
    # Convert based on target column type
    try:
        if 'INT' in column_type.upper():
            # For INT columns
            if isinstance(value, str):
                # Try to convert string to int
                try:
                    return int(value)
                except ValueError:
                    print(f"Cannot convert '{value}' to INT for {table_name}.{column_name}, using default")
                    return 0
            return int(value) if value is not None else None
        
        elif 'REAL' in column_type.upper() or 'FLOAT' in column_type.upper() or 'DECIMAL' in column_type.upper():
            # For floating-point columns
            if isinstance(value, str):
                try:
                    return float(value)
                except ValueError:
                    print(f"Cannot convert '{value}' to REAL for {table_name}.{column_name}, using default")
                    return 0.0
            return float(value) if value is not None else None
        
        elif 'TEXT' in column_type.upper() or 'VARCHAR' in column_type.upper() or 'CHAR' in column_type.upper():
            # For text columns
            return str(value) if value is not None else None
        
        elif 'DATE' in column_type.upper():
            # For date columns
            return value
        
        elif 'BIT' in column_type.upper():
            # For boolean columns
            if isinstance(value, bool):
                return 1 if value else 0
            elif isinstance(value, (int, float)):
                return 1 if value > 0 else 0
            elif isinstance(value, str):
                return 1 if value.lower() in ('true', 'yes', 'y', '1') else 0
            return 0
            
        else:
            # Default handling for other types
            return value
    except Exception as e:
        print(f"Error converting value '{value}' for {table_name}.{column_name}: {e}")
        return None

# Function to get column types from a table
def get_column_types(cursor, table_name):
    """Gets the data types of columns in a table."""
    column_types = {}
    cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'")
    for row in cursor.fetchall():
        column_types[row[0]] = row[1]
    return column_types

# Function to check if a column has a NOT NULL constraint
def get_nullable_columns(cursor, table_name):
    """Gets information about which columns are nullable."""
    nullable_info = {}
    cursor.execute(f"SELECT COLUMN_NAME, IS_NULLABLE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'")
    for row in cursor.fetchall():
        nullable_info[row[0]] = row[1] == 'YES'
    return nullable_info

def import_data():
    try:
        # Connect to SQL Server
        conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-HLMO0COA\\SQLEXPRESS;DATABASE=SDM_Project;Trusted_Connection=yes;')
        cursor = conn.cursor()
        
        print("Connected to SQL Server database")
        
        # Get list of tables in SQL Server
        cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
        sql_tables = [row[0] for row in cursor.fetchall()]
        print(f"Found {len(sql_tables)} tables in SQL Server database")
        
        # Disable constraints
        print("Disabling constraints...")
        cursor.execute("SELECT DISTINCT OBJECT_NAME(parent_object_id) FROM sys.foreign_keys")
        tables = [row[0] for row in cursor.fetchall()]
        for table in tables:
            cursor.execute(f"ALTER TABLE [{table}] NOCHECK CONSTRAINT ALL")
        conn.commit()
        
        # SQLite imports
        sqlite_files = [
            {"db": "aenc.sqlite", "tables": []},
            
        ]
        
        for file in sqlite_files:
            db_path = f"../../data/raw/{file['db']}"
            print(f"Processing SQLite database: {file['db']}")
            
            try:
                conn_sqlite = sql.connect(db_path)
                
                # Dynamically fetch all table names from the SQLite database
                cursor_sqlite = conn_sqlite.cursor()
                cursor_sqlite.execute("SELECT name FROM sqlite_master WHERE type='table'")
                tables = [row[0] for row in cursor_sqlite.fetchall()]
                cursor_sqlite.close()
                
                print(f"Found tables: {tables}")
                
                for table in tables:
                    print(f"Importing table: {table}")
                    
                    # Check if table exists in SQL Server
                    if table.upper() not in [t.upper() for t in sql_tables]:
                        print(f"Table {table} not found in SQL Server, skipping...")
                        continue
                    
                    # Get column information
                    column_types = get_column_types(cursor, table)
                    nullable_info = get_nullable_columns(cursor, table)
                    
                    # Read data from SQLite
                    df = pd.read_sql(f"SELECT * FROM {table}", conn_sqlite)
                    if df.empty:
                        print(f"No data found in {table}, skipping...")
                        continue
                    
                    # Get SQL Server columns
                    cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table}'")
                    sql_cols = [row[0] for row in cursor.fetchall()]
                    
                    # Find common columns
                    common_cols = []
                    for col in df.columns:
                        matching_cols = [s_col for s_col in sql_cols if s_col.upper() == col.upper()]
                        if matching_cols:
                            common_cols.append((col, matching_cols[0]))  # (sqlite_col, sql_server_col)
                    
                    if not common_cols:
                        print(f"No matching columns found for {table}, skipping...")
                        continue
                    
                    # Prepare SQL statement
                    sql_cols_str = ", ".join([f"[{col[1]}]" for col in common_cols])
                    placeholders = ", ".join(["?"] * len(common_cols))
                    
                    # Standard insert for tables
                    for _, row in df.iterrows():
                        try:
                            # Convert values
                            values = []
                            for sqlite_col, sql_col in common_cols:
                                value = convert_numpy_type(row[sqlite_col])
                                col_type = column_types.get(sql_col, 'TEXT')
                                converted_value = convert_value_for_column(value, sql_col, col_type, table.upper())
                                values.append(converted_value)
                            
                            # Execute insert
                            cursor.execute(f"INSERT INTO {table} ({sql_cols_str}) VALUES ({placeholders})", tuple(values))
                        except Exception as e:
                            print(f"Error inserting row in {table}: {e}")
                    
                    # Commit after each table
                    conn.commit()
                    print(f"Successfully imported table: {table}")
                
                conn_sqlite.close()
            except Exception as e:
                print(f"Error processing SQLite database {file['db']}: {e}")

        print("Re-enabling constraints...")
        for table in tables:
            cursor.execute(f"ALTER TABLE [{table}] WITH NOCHECK CHECK CONSTRAINT ALL")
        conn.commit()
        
        cursor.close()
        conn.close()
        print("Database import completed successfully")
        
    except Exception as e:
        print(f"Fatal error during import: {e}")
        raise

# Execute the import function
if __name__ == "__main__":
    try:
        import_data()
        print("Import completed successfully.")
    except Exception as e:
        print(f"Import failed: {e}")

Connected to SQL Server database
Found 30 tables in SQL Server database
Disabling constraints...
Processing SQLite database: aenc.sqlite
Found tables: ['Department', 'Product', 'State', 'Bonus', 'Customer', 'Employee', 'Sales_Order', 'Sales_Order_Item']
Importing table: Department
Successfully imported table: Department
Importing table: Product
Successfully imported table: Product
Importing table: State
Successfully imported table: State
Importing table: Bonus
Successfully imported table: Bonus
Importing table: Customer
Successfully imported table: Customer
Importing table: Employee
Successfully imported table: Employee
Importing table: Sales_Order
Successfully imported table: Sales_Order
Importing table: Sales_Order_Item
Successfully imported table: Sales_Order_Item
Re-enabling constraints...
Database import completed successfully
Import completed successfully.


Acces importeren


In [5]:
import os
import pyodbc
import pandas as pd
from tqdm import tqdm

# Define paths
access_db_path = os.path.abspath('../../data/raw/AdventureWorks.mdb')

# SQL Server connection string
sql_server_conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-HLMO0COA\\SQLEXPRESS;' 
    'DATABASE=SDM_project;' 
    'Trusted_Connection=yes;'
)

# Define the mapping between Access and SQL Server table names
table_mapping = {
    'HumanResources_Department': 'resources_department',
    'HumanResources_Employee': 'Employee',
    'Person_Address': 'Address',
    'Person_Person': 'person',
    'Production_BillOfMaterials': 'billOfMaterials',
    'Production_Product': 'Product',
    'Production_ProductCategory': 'productCategory',
    'Purchasing_PurchaseOrderDetail': 'purchaseOrderDetail',
    'Purchasing_PurchaseOrderHeader': 'purchaseOrderHeader',
    'Purchasing_Vendor': 'purchasing_vendor',
    'Sales_Customer': 'Customer',
    'Sales_SalesOrderDetail': 'Sales_OrderDetail',
    'Sales_SalesOrderHeader': 'Sales_OrderHeader',
    'Sales_SalesTerritory': 'Sales_territory',
    'Sales_Store': 'Sales_Store'
}

# Define column mappings for tables where field names differ
column_mappings = {
    # resources_department
    'resources_department': {
        'DepartmentID': 'DepartmentID',
        'Name': 'Name',
        'GroupName': 'GroupName',
        'ModifiedDate': 'ModifiedDate'
    },
    # Employee
    'Employee': {
        'BusinessEntityID': 'BusinessEntityID',
        'ss_number': 'NationalIDNumber',
        'LoginID': 'LoginID',
        'OrganizationNode': 'OrganizationNode',
        'OrganizationLevel': 'OrganizationLevel',
        'JobTitle': 'JobTitle',
        'BirthDate': 'BirthDate',
        'maritalstatus': 'MaritalStatus',
        'gender': 'Gender',
        'HireDate': 'HireDate',
        'salariedFlag': 'SalariedFlag',
        'vacationHours': 'VacationHours',
        'SickleaveHours': 'SickleaveHours',
        'currentFlag': 'CurrentFlag',
        'rowguid': 'rowguid',
        'modifiedDate': 'ModifiedDate',
        'departmentid': 'dept_id'
    },
    # Address
    'Address': {
        'AddressID': 'AddressID',
        'AddressLine1': 'AddressLine1',
        'AddressLine2': 'AddressLine2',
        'City': 'City',
        'StateProvinceID': 'StateProvinceID',
        'PostalCode': 'PostalCode',
        'SpatialLocation': 'SpatialLocation',
        'rowguid': 'rowguid',
        'ModifiedDate': 'ModifiedDate',
        'BusinessEntityID': 'BusinessEntityID'
    },
    # person
    'person': {
        'businessentityid': 'BusinessEntityID',
        'persontype': 'PersonType',
        'namestyle': 'NameStyle',
        'title': 'Title',
        'firstname': 'FirstName',
        'middlename': 'MiddleName',
        'lastname': 'LastName',
        'suffix': 'Suffix',
        'emailpromotion': 'EmailPromotion',
        'rowguid': 'rowguid',
        'modifieddate': 'ModifiedDate'
    },
    # billOfMaterials
    'billOfMaterials': {
        'BillOfMaterialsID': 'BillOfMaterialsID',
        'ProductAssemblyID': 'ProductAssemblyID',
        'ComponentID': 'ComponentID',
        'StartDate': 'StartDate',
        'EndDate': 'EndDate',
        'UnitMeasureCode': 'UnitMeasureCode',
        'BOMLevel': 'BOMLevel',
        'PerAssemblyQty': 'PerAssemblyQty',
        'ModifiedDate': 'ModifiedDate'
    },
    # Product
    'Product': {
        'id': 'ProductID',
        'Name': 'Name',
        'ProductNumber': 'ProductNumber',
        'MakeFlag': 'MakeFlag',
        'FinishedGoodsFlag': 'FinishedGoodsFlag',
        'Color': 'Color',
        'SafetyStockLevel': 'SafetyStockLevel',
        'ReorderPoint': 'ReorderPoint',
        'StandardCost': 'StandardCost',
        'ListPrice': 'ListPrice',
        'Size': 'Size',
        'SizeUnitMeasureCode': 'SizeUnitMeasureCode',
        'WeightUnitMeasureCode': 'WeightUnitMeasureCode',
        'Weight': 'Weight',
        'DaysToManufacture': 'DaysToManufacture',
        'ProductLine': 'ProductLine',
        'Class': 'Class',
        'Style': 'Style',
        'ProductSubcategoryID': 'ProductSubcategoryID',
        'productModelID': 'productModelID',
        'SellStartDate': 'SellStartDate',
        'SellEndDate': 'SellEndDate',
        'DiscontinuedDate': 'DiscontinuedDate',
        'rowguid': 'rowguid',
        'ModifiedDate': 'ModifiedDate',
        'productCategoryID': 'ProductCategoryID'
    },
    # productCategory
    'productCategory': {
        'productCategoryID': 'ProductCategoryID',
        'Name': 'Name',
        'rowguid': 'rowguid',
        'ModifiedDate': 'ModifiedDate'
    },
    # purchaseOrderDetail
    'purchaseOrderDetail': {
        'purchaseOrderId': 'PurchaseOrderID',
        'purchaseOrderDetailId': 'PurchaseOrderDetailID',
        'dueDate': 'DueDate',
        'orderQty': 'OrderQty',
        'productId': 'ProductID',
        'unitPrice': 'UnitPrice',
        'lineTotal': 'LineTotal',
        'receivedQty': 'ReceivedQty',
        'rejectedQty': 'RejectedQty',
        'stockedQty': 'StockedQty',
        'ModifiedDate': 'ModifiedDate'
    },
    # purchaseOrderHeader
    'purchaseOrderHeader': {
        'purchaseOrderId': 'PurchaseOrderID',
        'revisionNumber': 'RevisionNumber',
        'status': 'Status',
        'employeeID': 'EmployeeID',
        'vendorID': 'VendorID',
        'shipMethodID': 'ShipMethodID',
        'orderDate': 'OrderDate',
        'shipDate': 'ShipDate',
        'subtotal': 'SubTotal',
        'taxAmt': 'TaxAmt',
        'freight': 'Freight',
        'totalDue': 'TotalDue',
        'ModifiedDate': 'ModifiedDate'
    },
    # purchasing_vendor
    'purchasing_vendor': {
        'businessEntityId': 'BusinessEntityID',
        'accountNumber': 'AccountNumber',
        'name': 'Name',
        'creditRating': 'CreditRating',
        'preferredVendorStatus': 'PreferredVendorStatus',
        'ActiveFlag': 'ActiveFlag',
        'purchasingWebServiceURL': 'PurchasingWebServiceURL',
        'ModifiedDate': 'ModifiedDate',
    },
    # Customer
    'Customer': {
        'id': 'CustomerID',
        'PersonID': 'PersonID',
        'StoreID': 'StoreID',
        'AccountNumber': 'AccountNumber',
        'territoryID': 'TerritoryID',
        'rowguid': 'rowguid',
        'ModifiedDate': 'ModifiedDate'
    },
    # Sales_OrderDetail
    'Sales_OrderDetail': {
        'salesOrderId': 'SalesOrderID',
        'salesOrderDetailId': 'SalesOrderDetailID',
        'carrierTrackingNumber': 'CarrierTrackingNumber',
        'orderQty': 'OrderQty',
        'productId': 'ProductID',
        'specialOfferId': 'SpecialOfferID',
        'unitPrice': 'UnitPrice',
        'unitPriceDiscount': 'UnitPriceDiscount',
        'lineTotal': 'LineTotal',
        'rowguid': 'rowguid',
        'ModifiedDate': 'ModifiedDate'
    },
    # Sales_OrderHeader
    'Sales_OrderHeader': {
        'SalesOrderID': 'SalesOrderID',
        'revisionNumber': 'RevisionNumber',
        'orderDate': 'OrderDate',
        'dueDate': 'DueDate',
        'shipDate': 'ShipDate',
        'status': 'Status',
        'salesOrderNumber': 'SalesOrderNumber',
        'purchaseOrderNumber': 'PurchaseOrderNumber',
        'accountNumber': 'AccountNumber',
        'customerID': 'CustomerID',
        'territoryID': 'TerritoryID',
        'billToAddressID': 'BillToAddressID',
        'shipToAddressID': 'ShipToAddressID',
        'shipMethodID': 'ShipMethodID',
        'creditCardID': 'CreditCardID',
        'creditCardApprovalCode': 'CreditCardApprovalCode',
        'currencyRateID': 'CurrencyRateID',
        'subTotal': 'SubTotal',
        'taxAmt': 'TaxAmt',
        'freight': 'Freight',
        'totalDue': 'TotalDue',
        'comment': 'Comment',
        'rowguid': 'rowguid',
        'ModifiedDate': 'ModifiedDate'
    },
    # Sales_territory
    'Sales_territory': {
        'territoryID': 'TerritoryID',
        'name': 'Tame',
        'CountryRegionCode': 'CountryRegionCode',
        'GroupName': 'Group',
        'SalesYTD': 'SalesYTD',
        'SalesLastYear': 'SalesLastYear',
        'CostYTD': 'CostYTD',
        'CostLastYear': 'CostLastYear',
        'rowguid': 'rowguid',
        'ModifiedDate': 'ModifiedDate'
    },
    # Sales_Store
    'Sales_Store': {
        'BusinessEntityId': 'BusinessEntityId',
        'Name': 'Name',
        'SalesPersonId': 'SalesPersonId',
        'rowguid': 'rowguid',
        'ModifiedDate': 'ModifiedDate',
    }
}

# Function to get all tables from Access database
def get_access_tables(conn):
    cursor = conn.cursor()
    tables = []
    
    for row in cursor.tables(tableType='TABLE'):
        if row.table_name.startswith('MSys'):
            continue  # Skip system tables
        tables.append(row.table_name)
    
    return tables

# Function to disable constraints in SQL Server database
def disable_constraints(conn):
    cursor = conn.cursor()
    
    # Get all tables with constraints
    tables_query = """
    SELECT DISTINCT OBJECT_NAME(parent_object_id) AS TableName
    FROM sys.foreign_keys
    """
    
    try:
        cursor.execute(tables_query)
        tables_with_constraints = [row[0] for row in cursor.fetchall()]
        
        # Disable constraints for each table individually
        for table in tables_with_constraints:
            print(f"  Disabling constraints for {table}...")
            cursor.execute(f"ALTER TABLE [{table}] NOCHECK CONSTRAINT ALL")
            conn.commit()
        
        print("All constraints disabled")
    except Exception as e:
        print(f"Error disabling constraints: {str(e)}")
    
    return tables_with_constraints

# Function to enable constraints in SQL Server database
def enable_constraints(conn, tables_with_constraints):
    cursor = conn.cursor()
    
    try:
        # Re-enable constraints for each table individually
        for table in tables_with_constraints:
            print(f"  Re-enabling constraints for {table}...")
            cursor.execute(f"ALTER TABLE [{table}] WITH CHECK CHECK CONSTRAINT ALL")
            conn.commit()
        
        print("All constraints re-enabled successfully")
    except Exception as e:
        print(f"Error re-enabling constraints: {str(e)}")



# Function to transfer data from Access to SQL Server
def transfer_data(access_conn, sql_conn):
    access_cursor = access_conn.cursor()
    sql_cursor = sql_conn.cursor()
    
    # Get table names from Access
    access_tables = get_access_tables(access_conn)
    print(f"Found {len(access_tables)} tables in Access database")
    
    # Get table names from SQL Server
    sql_cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
    sql_tables = [row[0] for row in sql_cursor.fetchall()]
    print(f"Found {len(sql_tables)} tables in SQL Server database")
    
    # Match tables between the two databases using the mapping
    mapped_tables = []
    for access_table in access_tables:
        # First check if there's a specific mapping for this table
        if access_table in table_mapping:
            sql_table = table_mapping[access_table]
            if sql_table in sql_tables:
                mapped_tables.append((access_table, sql_table))
                continue
        
        # If no mapping or mapped table doesn't exist, try case-insensitive match
        matches = [sql_table for sql_table in sql_tables if sql_table.lower() == access_table.lower()]
        if matches:
            mapped_tables.append((access_table, matches[0]))
    
    print(f"Found {len(mapped_tables)} mapping tables between databases")
    
    # Process each matching table
    successful_transfers = 0
    for access_table, sql_table in tqdm(mapped_tables, desc="Transferring tables"):
        try:
            # Get column information from SQL Server table
            sql_cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{sql_table}'")
            sql_columns_info = {row[0].lower(): row[1] for row in sql_cursor.fetchall()}
            sql_columns = list(sql_columns_info.keys())
            
            # Get data from Access table
            access_cursor.execute(f"SELECT * FROM [{access_table}]")
            access_columns = [column[0].lower() for column in access_cursor.description]
            
            # Find matching columns using column mappings if available
            column_map = {}
            if sql_table in column_mappings:
                # Use defined column mappings
                for access_col in access_columns:
                    # Try to find a mapping for this column
                    mapped_col = None
                    for sql_col, access_col_name in column_mappings[sql_table].items():
                        if access_col.lower() == access_col_name.lower():
                            mapped_col = sql_col
                            break
                    
                    # If mapping found and mapped column exists in SQL table
                    if mapped_col and mapped_col.lower() in sql_columns:
                        column_map[access_col] = mapped_col
                    # Otherwise try direct match
                    elif access_col.lower() in sql_columns:
                        column_map[access_col] = access_col
            else:
                # If no explicit mapping, try direct case-insensitive match
                for access_col in access_columns:
                    for sql_col in sql_columns:
                        if access_col.lower() == sql_col.lower():
                            column_map[access_col] = sql_col
                            break
            
            # Get matching columns (Access columns that have a mapping to SQL columns)
            matching_columns = list(column_map.keys())
            
            print(f"  Table: {access_table} -> {sql_table}")
            print(f"  Mapped {len(matching_columns)} of {len(access_columns)} columns")
            
            # Prepare data for insertion
            rows = []
            for row in access_cursor.fetchall():
                # Create a dictionary with only the matching columns
                row_dict = {}
                for i, col in enumerate(access_columns):
                    if col in matching_columns:
                        row_dict[col] = row[i]
                rows.append(row_dict)
            
            if rows:
                # Prepare insert statement with mapped column names
                sql_cols = [column_map[col] for col in matching_columns]
                placeholders = ', '.join(['?' for _ in matching_columns])
                columns_str = ', '.join([f'[{col}]' for col in sql_cols])
                insert_query = f"INSERT INTO [{sql_table}] ({columns_str}) VALUES ({placeholders})"
                
                # Insert batch of rows
                success_count = 0
                for row_dict in rows:
                    values = [row_dict[col] for col in matching_columns]
                    try:
                        sql_cursor.execute(insert_query, values)
                        success_count += 1
                    except Exception as e:
                        print(f"  Error inserting into {sql_table}: {str(e)}")
                        print(f"  Query: {insert_query}")
                        print(f"  Values: {values}")
                        continue
                
                sql_conn.commit()
                successful_transfers += 1
                print(f"  Transferred {success_count} of {len(rows)} rows from {access_table} to {sql_table}")
            else:
                print(f"  No data found in {access_table}")
                
        except Exception as e:
            print(f"Error processing table {access_table}: {str(e)}")
    
    print(f"Successfully transferred data for {successful_transfers} out of {len(mapped_tables)} tables")

# Main function
def main():
    print("Starting data transfer from Access to SQL Server...")
    
    # Try to connect to Access database
    try:
        print(f"Connecting to Access database at {access_db_path}...")
        if not os.path.exists(access_db_path):
            print(f"Error: Access database file not found at {access_db_path}")
            return
        
        access_conn_str = (
            r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
            r"DBQ=" + access_db_path + ";"
        )
        access_conn = pyodbc.connect(access_conn_str)
        print("Connected to Access database successfully")
    except Exception as e:
        print(f"Error connecting to Access database: {str(e)}")
        print("\nAvailable ODBC drivers:")
        for driver in pyodbc.drivers():
            print(f"  - {driver}")
        return
    
    # Try to connect to SQL Server database
    try:
        print("Connecting to SQL Server database...")
        sql_conn = pyodbc.connect(sql_server_conn_str)
        print("Connected to SQL Server database successfully")
    except Exception as e:
        print(f"Error connecting to SQL Server database: {str(e)}")
        access_conn.close()
        return
    
    try:
        # Disable constraints
        tables_with_constraints = disable_constraints(sql_conn)
        
        # Transfer data
        transfer_data(access_conn, sql_conn)
        
        # Re-enable constraints
        enable_constraints(sql_conn, tables_with_constraints)
        
    except Exception as e:
        print(f"Error during data transfer: {str(e)}")
    
    finally:
        # Close connections
        print("Closing database connections...")
        access_conn.close()
        sql_conn.close()
        print("Data transfer process completed")

if __name__ == "__main__":
    main()

Starting data transfer from Access to SQL Server...
Connecting to Access database at c:\Users\veyse\source\DEDS_portfolio\data\raw\AdventureWorks.mdb...
Connected to Access database successfully
Connecting to SQL Server database...
Connected to SQL Server database successfully
  Disabling constraints for Bonus...
  Disabling constraints for Customer...
  Disabling constraints for CustomerCustomerDemo...
  Disabling constraints for EmployeeTerritories...
  Disabling constraints for OrderDetails...
  Disabling constraints for Orders...
  Disabling constraints for purchaseOrderDetail...
  Disabling constraints for purchaseOrderHeader...
  Disabling constraints for Sales_Order...
  Disabling constraints for sales_order_item...
  Disabling constraints for Sales_OrderDetail...
  Disabling constraints for Sales_OrderHeader...
  Disabling constraints for Sales_Store...
  Disabling constraints for Territories...
All constraints disabled
Found 16 tables in Access database
Found 30 tables in SQL 

Transferring tables:   0%|          | 0/15 [00:00<?, ?it/s]

  Table: HumanResources_Department -> resources_department
  Mapped 4 of 4 columns
  Transferred 16 of 16 rows from HumanResources_Department to resources_department
  Table: HumanResources_Employee -> Employee
  Mapped 16 of 17 columns


Transferring tables:  13%|█▎        | 2/15 [00:00<00:03,  3.31it/s]

  Transferred 290 of 290 rows from HumanResources_Employee to Employee
  Table: Person_Address -> Address
  Mapped 10 of 10 columns


...

Transferring tables:  93%|█████████▎| 14/15 [01:45<00:12, 12.03s/it]

  Transferred 31465 of 31465 rows from Sales_SalesOrderHeader to Sales_OrderHeader
  Table: Sales_SalesTerritory -> Sales_territory
  Mapped 10 of 10 columns
  Transferred 10 of 10 rows from Sales_SalesTerritory to Sales_territory
  Table: Sales_Store -> Sales_Store
  Mapped 5 of 5 columns


Transferring tables: 100%|██████████| 15/15 [01:45<00:00,  7.06s/it]

  Transferred 701 of 701 rows from Sales_Store to Sales_Store
Successfully transferred data for 15 out of 15 tables
  Re-enabling constraints for Bonus...
Error re-enabling constraints: ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The ALTER TABLE statement conflicted with the FOREIGN KEY constraint "FK__Bonus__emp_id__6EF57B66". The conflict occurred in database "SDM_project", table "dbo.Employee", column \'id\'. (547) (SQLExecDirectW)')
Closing database connections...


Data transfer process completed


Northwind(SQL Server) importeren


In [6]:
import pyodbc
from tqdm import tqdm

# SQL Server connection string for Northwind (source)
northwind_conn_str = (
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=LAPTOP-HLMO0COA\SQLEXPRESS;"
    r"DATABASE=northwind;"
    r"Trusted_Connection=yes;"
)

# SQL Server connection string for SDM_project (destination)
sdm_conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-HLMO0COA\\SQLEXPRESS;' 
    'DATABASE=SDM_project;' 
    'Trusted_Connection=yes;'
)

# Define the tables to transfer and their mapping to SDM_project tables
tables_to_transfer = {
    'Categories': 'Categories',
    'Customers': 'Customer',  # Note the different name in SDM_project
    'Employees': 'Employee',
    'Shippers': 'Shippers',
    'Suppliers': 'Suppliers',
    'Orders': 'Orders',
    'Products': 'Product',  # Note the different name in SDM_project
    '[Order Details]': 'OrderDetails',  # Note the different name in SDM_project
    'Region': 'Region',
    'Territories': 'Territories',
    'EmployeeTerritories': 'EmployeeTerritories'
}

# Define field mapping for tables with different column names or structures
field_mappings = {
    'Customers': {
        'CustomerID': 'id',  # Northwind CustomerID maps to id field in SDM_project
        'CompanyName': 'company_name',
        'ContactName': 'name',
        'Phone': 'phone',
        'Fax': 'fax',
        'Address': 'address',
        'City': 'city',
        'Region': 'state',
        'PostalCode': 'zip',
        'Country': 'Country'
    },
    'Employees': {
        'EmployeeID': 'emp_id',
        'LastName': 'emp_lname',
        'FirstName': 'emp_fname',
        'Title': 'JobTitle',
        'TitleOfCourtesy': 'title',
        'BirthDate': 'BirthDate',
        'HireDate': 'HireDate',
        'Address': None,  # No direct mapping
        'City': 'City',
        'Region': 'Region',
        'PostalCode': 'Postal-code',
        'Country': 'country',
        'HomePhone': 'home_phone',
        'Extension': 'extension',
        'Notes': 'Notes',
        'ReportsTo': 'ReportsTo',
        'PhotoPath': 'photopath'
    },
    'Products': {
        'ProductID': 'id',  # Northwind ProductID maps to id field in SDM_project
        'ProductName': 'Name',
        'SupplierID': 'SupplierID',
        'CategoryID': 'productCategoryID',
        'QuantityPerUnit': 'Description',
        'UnitPrice': 'UnitPrice',
        'UnitsInStock': 'UnitsInStock',
        'UnitsOnOrder': 'UnitsOnOrder',
        'ReorderLevel': 'ReorderLevel',
        'Discontinued': 'Discontinued'
    },
    'Orders': {
        'OrderID': 'OrderID',
        'CustomerID': None,  # Special handling needed - CustomerID in Northwind is string, in SDM_project is int
        'OrderDate': 'OrderDate',
        'RequiredDate': 'RequiredDate',
        'ShippedDate': 'ShippedDate'
    },
    '[Order Details]': {
        'OrderID': 'OrderID',
        'ProductID': 'ProductID',
        'UnitPrice': 'UnitPrice',
        'Quantity': 'Quantity',
        'Discount': 'Discount'
    }
}

# Function to disable constraints in SQL Server database
def disable_constraints(conn):
    cursor = conn.cursor()
    
    # Get all tables with constraints
    tables_query = """
    SELECT DISTINCT OBJECT_NAME(parent_object_id) AS TableName
    FROM sys.foreign_keys
    """
    
    try:
        cursor.execute(tables_query)
        tables_with_constraints = [row[0] for row in cursor.fetchall()]
        
        # Disable constraints for each table individually
        for table in tables_with_constraints:
            print(f"  Disabling constraints for {table}...")
            cursor.execute(f"ALTER TABLE [{table}] NOCHECK CONSTRAINT ALL")
            conn.commit()
        
        print("All constraints disabled")
    except Exception as e:
        print(f"Error disabling constraints: {str(e)}")
    
    return tables_with_constraints

# Function to enable constraints in SQL Server database
def enable_constraints(conn, tables_with_constraints):
    cursor = conn.cursor()
    
    try:
        # Re-enable constraints for each table individually
        for table in tables_with_constraints:
            print(f"  Re-enabling constraints for {table}...")
            cursor.execute(f"ALTER TABLE [{table}] WITH CHECK CHECK CONSTRAINT ALL")
            conn.commit()
        
        print("All constraints re-enabled successfully")
    except Exception as e:
        print(f"Error re-enabling constraints: {str(e)}")

# Function to check if a table exists
def table_exists(conn, table_name):
    cursor = conn.cursor()
    cursor.execute(f"""
        SELECT COUNT(*) 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_NAME = '{table_name.replace('[', '').replace(']', '')}'
    """)
    return cursor.fetchone()[0] > 0

# Function to copy table schema
def copy_table_schema(source_conn, dest_conn, table_name):
    print(f"Copying schema for table: {table_name}")
    
    source_cursor = source_conn.cursor()
    dest_cursor = dest_conn.cursor()
    
    # Get column information
    source_cursor.execute(f"""
        SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, 
               IS_NULLABLE, COLUMN_DEFAULT
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = '{table_name.replace('[', '').replace(']', '')}'
        ORDER BY ORDINAL_POSITION
    """)
    
    columns = source_cursor.fetchall()
    
    # Get primary key information
    source_cursor.execute(f"""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
        WHERE OBJECTPROPERTY(OBJECT_ID(CONSTRAINT_NAME), 'IsPrimaryKey') = 1
        AND TABLE_NAME = '{table_name.replace('[', '').replace(']', '')}'
    """)
    
    primary_keys = [row[0] for row in source_cursor.fetchall()]
    
    # Construct CREATE TABLE statement
    create_table_sql = f"CREATE TABLE {table_name} (\n"
    
    for i, col in enumerate(columns):
        col_name, data_type, max_length, is_nullable, default_val = col
        
        # Format column definition
        create_table_sql += f"    [{col_name}] {data_type}"
        
        # Add length for certain data types
        if max_length is not None and max_length != -1 and data_type.lower() in ('char', 'varchar', 'nchar', 'nvarchar'):
            create_table_sql += f"({max_length})"
        
        # Add NULL/NOT NULL
        if is_nullable == 'NO':
            create_table_sql += " NOT NULL"
        else:
            create_table_sql += " NULL"
        
        # Add default value if exists
        if default_val:
            create_table_sql += f" DEFAULT {default_val}"
        
        # Add comma if not the last column
        if i < len(columns) - 1:
            create_table_sql += ",\n"
    
    # Add primary key constraint if exists
    if primary_keys:
        pk_columns = ", ".join([f"[{pk}]" for pk in primary_keys])
        create_table_sql += f",\n    CONSTRAINT [PK_{table_name.replace('[', '').replace(']', '')}] PRIMARY KEY CLUSTERED ({pk_columns})"
    
    create_table_sql += "\n)"
    
    try:
        # Create the table in destination
        dest_cursor.execute(create_table_sql)
        dest_conn.commit()
        print(f"  Table {table_name} created successfully")
        return True
    except Exception as e:
        print(f"  Error creating table {table_name}: {str(e)}")
        print(f"  SQL: {create_table_sql}")
        return False

# Function to copy data between tables with field mapping
def copy_table_data(source_conn, dest_conn, source_table, dest_table):
    print(f"Copying data from {source_table} to {dest_table}")
    
    source_cursor = source_conn.cursor()
    dest_cursor = dest_conn.cursor()
    
    try:
        # Get column names from source
        source_cursor.execute(f"SELECT * FROM {source_table} WHERE 1=0")
        source_columns = [column[0] for column in source_cursor.description]
        
        # Get column names from destination
        dest_cursor.execute(f"SELECT * FROM {dest_table} WHERE 1=0")
        dest_columns = [column[0] for column in dest_cursor.description]
        dest_columns_lower = [col.lower() for col in dest_columns]
        
        # Check for identity columns in destination
        dest_cursor.execute(f"""
            SELECT COLUMN_NAME 
            FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = '{dest_table.replace('[', '').replace(']', '')}' 
            AND COLUMNPROPERTY(OBJECT_ID('{dest_table.replace('[', '').replace(']', '')}'), COLUMN_NAME, 'IsIdentity') = 1
        """)
        identity_columns = [row[0].lower() for row in dest_cursor.fetchall()]
        print(f"  Identity columns in {dest_table}: {identity_columns}")
        
        # Get primary key info for destination table
        dest_cursor.execute(f"""
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
            WHERE OBJECTPROPERTY(OBJECT_ID(CONSTRAINT_NAME), 'IsPrimaryKey') = 1
            AND TABLE_NAME = '{dest_table.replace('[', '').replace(']', '')}'
        """)
        pk_columns = [row[0] for row in dest_cursor.fetchall()]
        print(f"  Primary key columns in {dest_table}: {pk_columns}")
        
        # Determine columns to insert and their mapping
        mapped_columns = []
        target_columns = []
        custom_transforms = {}
        
        # Check if we have a specific mapping for this table
        if source_table in field_mappings:
            mapping = field_mappings[source_table]
            
            for src_col in source_columns:
                # Skip identity columns unless we explicitly want to include them
                if src_col in mapping:
                    dest_col = mapping[src_col]
                    if dest_col is not None:
                        # Check if destination column exists
                        if dest_col.lower() in dest_columns_lower:
                            mapped_columns.append(src_col)
                            target_columns.append(dest_col)
                        else:
                            print(f"  Warning: Mapped column {dest_col} not found in destination table")
                else:
                    # Try direct match for columns not in mapping
                    if src_col.lower() in dest_columns_lower:
                        idx = dest_columns_lower.index(src_col.lower())
                        mapped_columns.append(src_col)
                        target_columns.append(dest_columns[idx])
        else:
            # No specific mapping, try direct matches
            for src_col in source_columns:
                if src_col.lower() in dest_columns_lower:
                    idx = dest_columns_lower.index(src_col.lower())
                    mapped_columns.append(src_col)
                    target_columns.append(dest_columns[idx])
        
        # Handle special case for CustomerID in Orders table (string to int conversion)
        if source_table == 'Orders' and 'CustomerID' in source_columns:
            # For Orders table, we need to map CustomerID through the Customer table
            # First, create a mapping from Northwind CustomerID (string) to SDM_project CustomerID (int)
            source_cursor.execute("SELECT CustomerID FROM Customers")
            northwind_customer_ids = [row[0] for row in source_cursor.fetchall()]
            
            customer_id_map = {}
            for n_cust_id in northwind_customer_ids:
                # We want to find the Customer record where id = n_cust_id
                dest_cursor.execute(f"SELECT CustomerId FROM Customer WHERE id = '{n_cust_id}'")
                result = dest_cursor.fetchone()
                if result:
                    customer_id_map[n_cust_id] = result[0]
            
            custom_transforms['CustomerID'] = lambda x: customer_id_map.get(x, None) if x else None
            if 'CustomerID' not in mapped_columns:
                mapped_columns.append('CustomerID')
                target_columns.append('CustomerID')
                
        # Handle special case for ProductID in Order Details table
        if source_table == '[Order Details]' and 'ProductID' in source_columns:
            # Create a mapping from Northwind ProductID to SDM_project productId
            source_cursor.execute("SELECT ProductID FROM Products")
            northwind_product_ids = [row[0] for row in source_cursor.fetchall()]
            
            product_id_map = {}
            for n_prod_id in northwind_product_ids:
                # We want to find the Product record where id = n_prod_id
                dest_cursor.execute(f"SELECT productId FROM Product WHERE id = '{n_prod_id}'")
                result = dest_cursor.fetchone()
                if result:
                    product_id_map[n_prod_id] = result[0]
            
            custom_transforms['ProductID'] = lambda x: product_id_map.get(x, None) if x else None
        
        # Check if we have any columns to insert
        if not mapped_columns:
            print(f"  No matching columns found between {source_table} and {dest_table}")
            return False
        
        print(f"  Mapped columns: {', '.join(mapped_columns)} -> {', '.join(target_columns)}")
        
        # Get data from source
        mapped_cols_str = ", ".join([f"[{col}]" for col in mapped_columns])
        source_cursor.execute(f"SELECT {mapped_cols_str} FROM {source_table}")
        rows = source_cursor.fetchall()
        
        if not rows:
            print(f"  No data found in {source_table}")
            return True
        
        # If any target column is an identity column, enable identity insert
        identity_insert_needed = any(col.lower() in identity_columns for col in target_columns)
        if identity_insert_needed:
            try:
                dest_cursor.execute(f"SET IDENTITY_INSERT {dest_table} ON")
                print(f"  IDENTITY_INSERT enabled for {dest_table}")
            except Exception as e:
                print(f"  Warning: Failed to enable IDENTITY_INSERT: {str(e)}")
        
        # Prepare insert statement
        target_cols_str = ", ".join([f"[{col}]" for col in target_columns])
        placeholders = ", ".join(["?" for _ in target_columns])
        insert_sql = f"INSERT INTO {dest_table} ({target_cols_str}) VALUES ({placeholders})"
        
        # Insert batch of rows
        success_count = 0
        skipped_count = 0
        
        for row in tqdm(rows, desc=f"  Inserting into {dest_table}"):
            # Apply any custom transformations
            transformed_row = list(row)
            for i, col in enumerate(mapped_columns):
                if col in custom_transforms:
                    transformed_row[i] = custom_transforms[col](row[i])
            
            # Skip rows with null values for required columns
            if any(transformed_row[i] is None and target_columns[i] in pk_columns for i in range(len(transformed_row))):
                skipped_count += 1
                continue
            
            # Check if row already exists (if table has primary keys)
            if pk_columns:
                pk_where = []
                pk_values = []
                
                for i, col in enumerate(target_columns):
                    if col in pk_columns:
                        if transformed_row[i] is None:
                            pk_where.append(f"[{col}] IS NULL")
                        else:
                            pk_where.append(f"[{col}] = ?")
                            pk_values.append(transformed_row[i])
                
                if pk_where:
                    check_sql = f"SELECT COUNT(*) FROM {dest_table} WHERE {' AND '.join(pk_where)}"
                    try:
                        dest_cursor.execute(check_sql, pk_values)
                        exists = dest_cursor.fetchone()[0] > 0
                        
                        if exists:
                            skipped_count += 1
                            continue
                    except Exception as e:
                        print(f"  Error checking for existing row: {str(e)}")
                        skipped_count += 1
                        continue
            
            # Execute the insert
            try:
                dest_cursor.execute(insert_sql, transformed_row)
                success_count += 1
            except Exception as e:
                print(f"  Error inserting row: {str(e)}")
                skipped_count += 1
        
        # Turn off identity insert if we turned it on
        if identity_insert_needed:
            try:
                dest_cursor.execute(f"SET IDENTITY_INSERT {dest_table} OFF")
                print(f"  IDENTITY_INSERT disabled for {dest_table}")
            except Exception as e:
                print(f"  Warning: Failed to disable IDENTITY_INSERT: {str(e)}")
        
        dest_conn.commit()
        print(f"  Transferred {success_count} new rows to {dest_table} (Skipped {skipped_count} rows)")
        return True
    
    except Exception as e:
        print(f"  Error copying data from {source_table} to {dest_table}: {str(e)}")
        return False

# Function to add foreign key constraints
def add_foreign_keys(source_conn, dest_conn):
    print("Adding foreign key constraints...")
    
    source_cursor = source_conn.cursor()
    dest_cursor = dest_conn.cursor()
    
    try:
        # Get foreign key information from source
        source_cursor.execute("""
            SELECT 
                fk.name AS FKName,
                OBJECT_NAME(fk.parent_object_id) AS TableName,
                COL_NAME(fkc.parent_object_id, fkc.parent_column_id) AS ColumnName,
                OBJECT_NAME(fk.referenced_object_id) AS ReferenceTableName,
                COL_NAME(fkc.referenced_object_id, fkc.referenced_column_id) AS ReferenceColumnName
            FROM 
                sys.foreign_keys AS fk
                INNER JOIN sys.foreign_key_columns AS fkc ON fk.object_id = fkc.constraint_object_id
            ORDER BY
                TableName, ColumnName
        """)
        
        foreign_keys = source_cursor.fetchall()
        
        for fk in foreign_keys:
            fk_name, table_name, column_name, ref_table_name, ref_column_name = fk
            
            # Check if both tables exist in destination
            if not table_exists(dest_conn, table_name) or not table_exists(dest_conn, ref_table_name):
                print(f"  Skipping foreign key {fk_name}: Table(s) not found")
                continue
            
            try:
                # Create foreign key constraint
                fk_sql = f"""
                ALTER TABLE [{table_name}] WITH CHECK ADD CONSTRAINT [{fk_name}]
                FOREIGN KEY ([{column_name}]) REFERENCES [{ref_table_name}] ([{ref_column_name}])
                """
                
                dest_cursor.execute(fk_sql)
                dest_conn.commit()
                print(f"  Added foreign key {fk_name}")
            
            except Exception as e:
                print(f"  Error adding foreign key {fk_name}: {str(e)}")
    
    except Exception as e:
        print(f"Error getting foreign key information: {str(e)}")

# Main function
def main():
    print("Starting Northwind to SDM_project data transfer...")
    
    # Connect to source database (Northwind)
    try:
        print("Connecting to Northwind database...")
        source_conn = pyodbc.connect(northwind_conn_str)
        print("Connected to Northwind database successfully")
    except Exception as e:
        print(f"Error connecting to Northwind database: {str(e)}")
        return
    
    # Connect to destination database (SDM_project)
    try:
        print("Connecting to SDM_project database...")
        dest_conn = pyodbc.connect(sdm_conn_str)
        print("Connected to SDM_project database successfully")
    except Exception as e:
        print(f"Error connecting to SDM_project database: {str(e)}")
        source_conn.close()
        return
    
    try:
        # Disable constraints in destination
        tables_with_constraints = disable_constraints(dest_conn)
        
        # Process base tables first (those without foreign key dependencies)
        base_tables = ['Categories', 'Customers', 'Employees', 'Shippers', 'Suppliers', 'Region']
        dependent_tables = ['Products', 'Orders', 'Territories', '[Order Details]', 'EmployeeTerritories']
        
        # Process base tables first
        print("\nProcessing base tables...")
        for source_table in base_tables:
            if source_table in tables_to_transfer:
                dest_table = tables_to_transfer[source_table]
                copy_table_data(source_conn, dest_conn, source_table, dest_table)
        
        # Process dependent tables
        print("\nProcessing dependent tables...")
        for source_table in dependent_tables:
            if source_table in tables_to_transfer:
                dest_table = tables_to_transfer[source_table]
                copy_table_data(source_conn, dest_conn, source_table, dest_table)
        
        # Re-enable constraints
        enable_constraints(dest_conn, tables_with_constraints)
        
        print("Data transfer completed successfully")
    
    except Exception as e:
        print(f"Error during data transfer: {str(e)}")
    
    finally:
        # Close connections
        source_conn.close()
        dest_conn.close()
        print("Database connections closed")

if __name__ == "__main__":
    main()

Starting Northwind to SDM_project data transfer...
Connecting to Northwind database...
Connected to Northwind database successfully
Connecting to SDM_project database...
Connected to SDM_project database successfully
  Disabling constraints for Bonus...
  Disabling constraints for Customer...
  Disabling constraints for CustomerCustomerDemo...
  Disabling constraints for EmployeeTerritories...
  Disabling constraints for OrderDetails...
  Disabling constraints for Orders...
  Disabling constraints for purchaseOrderDetail...
  Disabling constraints for purchaseOrderHeader...
  Disabling constraints for Sales_Order...
  Disabling constraints for sales_order_item...
  Disabling constraints for Sales_OrderDetail...
  Disabling constraints for Sales_OrderHeader...
  Disabling constraints for Sales_Store...
  Disabling constraints for Territories...
All constraints disabled

Processing base tables...
Copying data from Categories to Categories
  Identity columns in Categories: []
  Primary ke

  Inserting into Categories: 100%|██████████| 8/8 [00:00<00:00, 324.18it/s]

  Transferred 8 new rows to Categories (Skipped 0 rows)
Copying data from Customers to Customer


  Identity columns in Customer: ['customerid']
  Primary key columns in Customer: ['CustomerId']
  Mapped columns: CustomerID, CompanyName, ContactName, Address, City, Region, PostalCode, Country, Phone, Fax -> id, company_name, name, address, city, state, zip, Country, phone, fax


  Inserting into Customer: 100%|██████████| 91/91 [00:00<00:00, 353.56it/s]


...

  Transferred 53 new rows to Territories (Skipped 0 rows)
Copying data from [Order Details] to OrderDetails
  Identity columns in OrderDetails: []
  Primary key columns in OrderDetails: ['OrderID', 'ProductID']
  Mapped columns: OrderID, ProductID, UnitPrice, Quantity, Discount -> OrderID, ProductID, UnitPrice, Quantity, Discount


  Inserting into OrderDetails: 100%|██████████| 2155/2155 [00:01<00:00, 1316.51it/s]


  Transferred 2155 new rows to OrderDetails (Skipped 0 rows)
Copying data from EmployeeTerritories to EmployeeTerritories
  Identity columns in EmployeeTerritories: []
  Primary key columns in EmployeeTerritories: ['EmployeeID', 'TerritoryID']
  Mapped columns: EmployeeID, TerritoryID -> EmployeeID, TerritoryID


  Inserting into EmployeeTerritories: 100%|██████████| 49/49 [00:00<00:00, 1253.04it/s]

  Transferred 49 new rows to EmployeeTerritories (Skipped 0 rows)
  Re-enabling constraints for Bonus...
Error re-enabling constraints: ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The ALTER TABLE statement conflicted with the FOREIGN KEY constraint "FK__Bonus__emp_id__6EF57B66". The conflict occurred in database "SDM_project", table "dbo.Employee", column \'id\'. (547) (SQLExecDirectW)')
Data transfer completed successfully
Database connections closed


SDM-> DWH


In [4]:
import pyodbc
import datetime

def log(message):
    """Print a timestamped log message"""
    print(f"[{datetime.datetime.now()}] {message}")

def main():
    # Connection info
    server = 'LAPTOP-HLMO0COA\\SQLEXPRESS'
    sdm_db = 'SDM_project'
    dwh_db = 'DWH_project'
    
    # Connect to both databases
    conn_string = f'DRIVER={{SQL Server}};SERVER={server};Trusted_Connection=yes;'
    sdm_conn = pyodbc.connect(conn_string + f'DATABASE={sdm_db}')
    dwh_conn = pyodbc.connect(conn_string + f'DATABASE={dwh_db}')
    
    log(f"Connected to both databases on {server}")
    
    # Get list of tables from SDM
    sdm_cursor = sdm_conn.cursor()
    sdm_cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
    tables = [row.TABLE_NAME for row in sdm_cursor.fetchall()]
    
    log(f"Found {len(tables)} tables to process")
    
    # Disable constraints in DWH
    dwh_cursor = dwh_conn.cursor()
    try:
        dwh_cursor.execute("EXEC sp_msforeachtable 'ALTER TABLE ? NOCHECK CONSTRAINT all'")
        dwh_conn.commit()
        log("Disabled all constraints")
    except Exception as e:
        log(f"Error disabling constraints: {str(e)}")
    
    # Process each table
    for i, table in enumerate(tables):
        try:
            log(f"Processing table {i+1}/{len(tables)}: {table}")
            
            # Get data from SDM
            sdm_cursor.execute(f"SELECT * FROM [{table}]")
            rows = sdm_cursor.fetchall()
            
            if not rows:
                log(f"Table {table} is empty, skipping")
                continue
                
            # Get column names
            columns = [column[0] for column in sdm_cursor.description]
            
            # Clear existing data in DWH
            dwh_cursor.execute(f"DELETE FROM [{table}]")
            dwh_conn.commit()
            
            # Prepare insert statement
            placeholders = ','.join(['?' for _ in columns])
            column_names = ','.join([f"[{col}]" for col in columns])
            insert_sql = f"INSERT INTO [{table}] ({column_names}) VALUES ({placeholders})"
            
            # Insert data into DWH
            counter = 0
            batch_size = 1000
            
            for batch_start in range(0, len(rows), batch_size):
                batch_end = min(batch_start + batch_size, len(rows))
                batch = rows[batch_start:batch_end]
                
                dwh_cursor.executemany(insert_sql, batch)
                dwh_conn.commit()
                
                counter += len(batch)
                if counter % 10000 == 0:
                    log(f"  - Inserted {counter} rows into {table}")
            
            log(f"Completed {table}: {counter} rows transferred")
            
        except Exception as e:
            log(f"Error processing table {table}: {str(e)}")
    
    # Special handling for Customer table - merge with CustomerDemographics
    try:
        log("Processing Customer table with demographics")
        
        # Clear existing data
        dwh_cursor.execute("DELETE FROM Customer")
        dwh_conn.commit()
        
        # Create a query that joins the customer tables
        merge_query = """
        SELECT 
            c.*,
            cd.CustomerTypeID,
            cd.CustomerDesc
        FROM 
            Customer c
        LEFT JOIN 
            CustomerCustomerDemo ccd ON c.CustomerId = ccd.CustomerID
        LEFT JOIN 
            CustomerDemographics cd ON ccd.CustomerTypeID = cd.CustomerTypeID
        """
        
        # Get merged data
        sdm_cursor.execute(merge_query)
        rows = sdm_cursor.fetchall()
        
        if rows:
            # Get column names
            columns = [column[0] for column in sdm_cursor.description]
            
            # Prepare insert statement
            placeholders = ','.join(['?' for _ in columns])
            column_names = ','.join([f"[{col}]" for col in columns])
            insert_sql = f"INSERT INTO Customer ({column_names}) VALUES ({placeholders})"
            
            # Insert data
            dwh_cursor.executemany(insert_sql, rows)
            dwh_conn.commit()
            
            log(f"Completed Customer table: {len(rows)} rows transferred with demographics")
    except Exception as e:
        log(f"Error processing Customer table: {str(e)}")
    
    # Re-enable constraints
    try:
        dwh_cursor.execute("EXEC sp_msforeachtable 'ALTER TABLE ? WITH CHECK CHECK CONSTRAINT all'")
        dwh_conn.commit()
        log("Re-enabled all constraints")
    except Exception as e:
        log(f"Error re-enabling constraints: {str(e)}")
    
    # Close connections
    sdm_conn.close()
    dwh_conn.close()
    log("ETL process completed")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Error: {str(e)}")

[2025-04-09 20:23:50.995823] Connected to both databases on LAPTOP-HLMO0COA\SQLEXPRESS
[2025-04-09 20:23:51.017168] Found 30 tables to process
[2025-04-09 20:23:51.091435] Disabled all constraints
[2025-04-09 20:23:51.091567] Processing table 1/30: Address
[2025-04-09 20:23:58.644977]   - Inserted 10000 rows into Address
[2025-04-09 20:24:04.096525] Completed Address: 19614 rows transferred
[2025-04-09 20:24:04.096810] Processing table 2/30: billOfMaterials
[2025-04-09 20:24:05.612267] Completed billOfMaterials: 2679 rows transferred
[2025-04-09 20:24:05.612551] Processing table 3/30: Bonus
[2025-04-09 20:24:05.640143] Completed Bonus: 37 rows transferred
[2025-04-09 20:24:05.640227] Processing table 4/30: Categories
[2025-04-09 20:24:05.669447] Completed Categories: 8 rows transferred
[2025-04-09 20:24:05.669514] Processing table 5/30: Customer
[2025-04-09 20:24:12.691785]   - Inserted 10000 rows into Customer
[2025-04-09 20:24:19.217311]   - Inserted 20000 rows into Customer
[2025-04